In [19]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
import time
import datetime
from pathlib import Path
from ipywidgets import Output, Button, Layout, HBox
from IPython.display import Image, display, clear_output

class LabelingTool:
    def __init__(self, classes: list, path: str) -> None:
        self.classes = classes
        self.path = Path(path)
        self.images = [f.name for f in self.path.glob("*.jpg")]
        self.labels = {}
        self.position = 0

    def _next_image(self, *args) -> None:
        """Select the next image and update the displays."""
        self.position += 1
        if self.position == len(self.images):
            self.position = 0

        # refresh display
        image = Image(self.path / self.images[self.position])
        with self.frame:
            clear_output(wait=True)
            display(image)

    def _previous_image(self, *args) -> None:
        """Select the previous image and update the displays."""
        self.position -= 1
        if self.position == -1:
            self.position = len(self.images) - 1

        # refresh display
        image = Image(self.path / self.images[self.position])
        with self.frame:
            clear_output(wait=True)
            display(image)

    def _store_label(self, button: Button) -> None:
        """Annotates the current image with the button's description."""
        # store label
        current_image = self.images[self.position]
        self.labels[current_image] = button.description

        # move on
        self._next_image()
    
    def start(self) -> None:
        image = Image(self.path / self.images[self.position])
        self.frame = Output(layout=Layout(height="300px", max_width="300px"))
        with self.frame:
            display(image)

        # navigation buttons
        backward_button = Button(description="< go back")
        backward_button.on_click(self._previous_image)
        forward_button = Button(description="next >")
        forward_button.on_click(self._next_image)
        self.navigation_buttons = [backward_button, forward_button]

        # class label buttons
        self.class_buttons = []
        for label in self.classes:
            label_button = Button(description=label)
            label_button.on_click(self._store_label)
            self.class_buttons.append(label_button)

        # display contents
        display(self.frame)
        display(HBox(self.navigation_buttons))
        display(HBox(self.class_buttons))

In [20]:
tool = LabelingTool(path="wiki_crop/00", classes = ["smile", "no smile"])

tool.start()

Output(layout=Layout(height='300px', max_width='300px'))